In [19]:
import numpy as np
import torch
import torch.nn as nn

### 互相关运算

In [17]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w] * K).sum()
    
    return Y


In [15]:
X = torch.arange(9.).reshape(3, 3)
X

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [16]:
K = torch.arange(4.).reshape(2, 2)
K

tensor([[0., 1.],
        [2., 3.]])

In [18]:
corr2d(X, K)

tensor([[19., 25.],
        [37., 43.]])

### 实现二维卷积层

In [22]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        # weight,bias为可学习的参数
        self.weight = nn.parameter(torch.rand(kernel_size))
        self.bias = nn.parameter(torch.zeros(1))
        
    def forward(self, x):
        return corr2d(x, self.weight) + self.bias # x和weight进行互相关运算再加偏移

### 卷积层简单应用: 检测图像中不同颜色的边缘

In [23]:
X = torch.ones((6, 8))
X[:, 2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [28]:
K = torch.tensor([[-1., 1.]])  # 注意中括号
K.shape

torch.Size([1, 2])

In [29]:
Y = corr2d(X, K)
Y  # 1代表白到黑边缘, -1代表黑到白边缘

tensor([[ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.],
        [ 0., -1.,  0.,  0.,  0.,  1.,  0.]])

### 学习由X生成Y的卷积核

In [33]:
conv2d = nn.Conv2d(1, 1, kernel_size=(1, 2), bias=False)
# 所有框架conv2d的输入都是4D
X = X.reshape(1, 1, 6, 8)
Y = Y.reshape(1, 1, 6, 7)

for i in range(10):
    Y_hat = conv2d(X)
    loss = (Y_hat - Y)**2
    conv2d.zero_grad()
    loss.sum().backward()
    conv2d.weight.data[:] -= 3e-02 * conv2d.weight.grad  # learning_rate=3e-02
    if (i + 1) % 2 == 0:
        print(f'batch{i+1}, loss {loss.sum():.3f}')

batch2, loss 4.456
batch4, loss 1.312
batch6, loss 0.451
batch8, loss 0.170
batch10, loss 0.067


In [35]:
conv2d.weight.data.reshape((1, 2))

tensor([[-0.9660,  1.0189]])